## Task

https://en.wikipedia.org/wiki/Nonogram

Given riddle, return possible coloring.

### Solution idea
This solution idea is to use inferencing and we know that inferencing is sufficient method.

#### Inferencing
We generate all possible colorings for rows and columns given requirements for them. We try to find fields which are always white or black and use this information further. 

For example:

When we find out that in every coloring for row one field is always black, then we go to column that also contatins this field and remove all colorings where this field is empty. Then we try to inference something about this column.

### Input
```
20 20
7 1
1 1 2
2 1 2
1 2 2
4 2 3
3 1 4
3 1 3
2 1 4
2 9
2 1 5
2 7
14
8 2
6 2 2
2 8 1 3
1 5 5 2
1 3 2 4 1
3 1 2 4 1
1 1 3 1 3
2 1 1 2
1 1 1 2
3 1 2 1 1
1 4 2 1 1
1 3 2 4
1 4 6 1
1 11 1
5 1 6 2
14
7 2
7 2
6 1 1
9 2
3 1 1 1
3 1 3
2 1 3
2 1 5
3 2 2
3 3 2
2 3 2
2 6
```
[number of rows] [number of columns]

[blocks in row_1]

[blocks in row_2]

$\vdots$

[blocks in column_1]

[blocks in column_2]

$\vdots$

### Output
Solved riddle

`1` for black field

`0` for empty field

## Imports

In [1]:
import numpy as np

## Code

In [2]:
# input

def read_data(file_name='zad_input.txt'):
    data = open('zad_input.txt', 'r')
    data = data.readlines()
    line0 = data[0].split()
    height = int(line0[0])
    width = int(line0[1])
    rows = []
    cols = []
    for i in range(1,height+1):
        specification = data[i].strip().split()
        length = len(specification)
        rows.append(np.zeros(length,dtype=np.int16))
        j=0
        for elem in specification:
            rows[i-1][j]=int(elem)
            j += 1
    for i in range(height+1, 1+height+width):
        specification = data[i].strip().split()
        length = len(specification)
        cols.append(np.zeros(length,dtype=np.int16))
        j=0
        for elem in specification:
            cols[i-height-1][j]=int(elem)
            j+=1
    return height,width,rows,cols

In [3]:
def prog(file_name='zad_input.txt'):
    height,width,rows,cols = read_data(file_name=file_name)
    
    print(solve(height,width,rows,cols))

In [4]:
def solve(height,width,rows,cols):
    good_rows = np.zeros(height,dtype=np.int8)
    good_cols = np.zeros(width,dtype=np.int8)
    
    board = np.full([height,width],-1,dtype=np.int8)
    
    colorings_rows = create_colorings_for_rows_cols(width,rows)
    colorings_cols = create_colorings_for_rows_cols(height,cols)
    
    dict_rows = create_dict(height)
    dict_cols = create_dict(width)
    flag_rows = True
    
    for i in range(height):
        sure = inference(colorings_rows[i])
        for j,elem in sure:
            if board[i,j]==-1:
                board[i,j] = elem
                dict_cols[j].add((i,elem))
    dict_rows = create_dict(height)
    
    for j in range(width):
        colorings_cols[j] = cut(colorings_cols[j],dict_cols[j])
        sure = inference(colorings_cols[j])
        for i,elem in sure:
            if board[i,j]==-1:
                board[i,j] = elem
                dict_rows[i].add((j,elem))
    dict_cols = create_dict(width)
    
    while not solved(board):
        if flag_rows:
            for i in range(height):
                if not is_empty_set(dict_rows[i]):
                    colorings_rows[i] = cut(colorings_rows[i],dict_rows[i])
                    sure = inference(colorings_rows[i])
                    for j,elem in sure:
                        if board[i,j]==-1:
                            board[i,j] = elem
                            dict_cols[j].add((i,elem))
            dict_rows = create_dict(height)
        else:
            for j in range(width):
                if not is_empty_set(dict_cols[j]):
                    colorings_cols[j] = cut(colorings_cols[j],dict_cols[j])
                    sure = inference(colorings_cols[j])
                    for i,elem in sure:
                        if board[i,j]==-1:
                            board[i,j] = elem
                            dict_rows[i].add((j,elem))
            dict_cols = create_dict(width)
        flag_rows = not flag_rows
    return board

def create_dict(size):
    d = {}
    for i in range(size):
        d[i] = set()
    return d

def solved(board):
    return np.all(board!=-1)


In [5]:
def create_colorings_for_rows_cols(size,specs_list):
    colorings_r_c = []
    for specs in specs_list:
        colorings_r_c.append(generate_coloring(size,specs))
    return colorings_r_c

def generate_coloring(size,specs):
    
    def rec_gen_col(i,line,specs,sum_,colorings):
        for j in range(i,np.size(line)-sum_+1):
            line_next = line.copy()
            line_next[j:j+specs[0]].fill(1)
            if np.size(specs)==1:
                colorings.append(line_next)
            else:
                rec_gen_col(j+specs[0]+1,line_next,specs[1:],sum_-specs[0]-1,colorings)
    
    colorings = []
    line = np.zeros(size,dtype=np.int8)
    
    if specs[0]==0:
        colorings.append(line)
    else:
        sum_ = np.size(specs)-1+np.sum(specs)
        rec_gen_col(0,line,specs,sum_,colorings)
    
    return colorings

In [6]:
def inference(colorings):
    size = np.size(colorings[0])
    sure = set()
    for i in range(size):
        elem = colorings[0][i]
        flag = True
        for coloring in colorings[1:]:
            if coloring[i]!=elem:
                flag = False
                break
        if flag==True:
            sure.add((i,elem))
    return sure

def cut(colorings,sure):
    for i,elem in sure:
        new_colorings = []
        for coloring in colorings:
            if coloring[i]==elem:
                new_colorings.append(coloring)
        colorings = new_colorings
    return colorings

In [7]:
def is_empty_set(s):
    return s == set()

In [8]:
prog()

[[1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0]
 [1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0]
 [0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 0 0 0 0 0]
 [1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 1 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0]
 [0 0 0 0 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 0]
 [0 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1]
 [1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1]
 [1 0 0 1 1 1 0 1 1 0 0 0 0 1 1 1 1 0 0 1]
 [0 1 1 1 0 0 1 0 1 1 0 0 0 0 1 1 1 1 0 1]
 [0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1]
 [0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1]]
